In [1]:
import pandas as pd
import numpy as np
import unidecode
import re

In [2]:
# importar os dados csv para pandas dataframe
cvd_2020_1 = pd.read_csv('dados/2020_1.csv',delimiter=';')
cvd_2020_2 = pd.read_csv('dados/2020_2.csv',delimiter=';')
cvd_2021_1 = pd.read_csv('dados/2020_2.csv',delimiter=';')
cvd_2021_2 = pd.read_csv('dados/2020_2.csv',delimiter=';')
cvd_2022_1 = pd.read_csv('dados/2020_2.csv',delimiter=';')

#### Explorando e Tratando os Dados da Covid do período 2020.1

In [3]:
# identificar porcentagem dos valores propriamente categorizados como NaN para cada coluna
for collumn in list(cvd_2020_1.columns):
    porcentagem = ((cvd_2020_1[collumn].isna().sum()/len(cvd_2020_1[collumn]))*100)
    print(collumn,": ",porcentagem)

regiao :  0.0
estado :  0.0
municipio :  0.0
coduf :  0.0
codmun :  0.0
codRegiaoSaude :  0.0
nomeRegiaoSaude :  0.0
data :  0.0
semanaEpi :  0.0
populacaoTCU2019 :  0.0
casosAcumulado :  0.0
casosNovos :  0.0
obitosAcumulado :  0.0
obitosNovos :  0.0
Recuperadosnovos :  0.0
emAcompanhamentoNovos :  0.0
interior/metropolitana :  1.0045230125054578


**Foi identificado valores faltantes na variável 'interior/metropolitana', sendo esses preenchidos com valor NaN. Para resolver isso será realizado imputação de valores com base na variável município, ou seja se a cidade localizada na variável for do interior a variável 'interior/metropolitana' receberá 0 senão receberá 1. Abaixo segue a lista das cidades metrópoles mapeadas visando identificar no dataset as mesmas, sendo que para isso será feita transformações no dado em relação a formatação das strings**

In [4]:
metropolitanas = ['Goiânia','Cuiabá','Macapá','Manaus','Belém','Santarém','Porto Velho','Boa Vista',
                  'Caracaraí','Caroebe','Palmas','Gurupi','Vitória','Belo Horizonte','Rio de Janeiro',
                  'Santos','Campinas','Jundiaí','Piracicaba','Ribeirão Preto','São José do Rio Preto',
                  'São Paulo','Sorocaba','São José dos Campos','Arapiraca','São Miguel dos Campos',
                  'Maceió','Santana do Ipanema','Palmeira dos Índios','Penedo','Delmiro Gouveia',
                  'Viçosa','União dos Palmares','Feira de Santana','Salvador','Juazeiro do Norte',
                  'Fortaleza','Sobral','São Luís','Imperatriz','Araruna','Barra de Santa Rosa',
                  'Cajazeiras','Campina Grande','Esperança','Guarabira','Itabaiana','João Pessoa',
                  'Patos','Sousa','Mamanguape','Piancó','Recife','Natal','Aracaju','Apucarana',
                  'Campo Mourão','Cascavel','Curitiba','Londrina','Maringá','	Toledo','Umuarama',
                  'Rio do Sul','Criciúma','Chapecó','Joaçaba','São Miguel do Oeste','Florianópolis',
                  'Itajaí','Lages','Joinville','Tubarão','Blumenau','Porto Alegre','Caxias do Sul']
for value in range(len(metropolitanas)):
    metropolitanas[value] = re.sub('\W+','',unidecode.unidecode(metropolitanas[value].lower()))

**Abaixo será verificado qual valor binário da variável 'interior/metropolitana' está associado a cidades metropolitanas, para isso utilizamos uma amostra do dataset somente de cidades metropolitanas selecionadas, se a divisão da soma dos valores da viaríavel 'interior/metropolitana' pelo tamanho da amostra for igual a 1, iremos inferir 1 a todos os valores onde o município se localize em zona metropolitana**

In [5]:
# identificamos que 1 é para metrópoles e 0 para cidades do interior
count_metropolis = cvd_2020_1[(cvd_2020_1['municipio'] == 'Salvador') | (cvd_2020_1['municipio'] == 'São Paulo') | (cvd_2020_1['municipio'] == 'Goiânia')]
len(count_metropolis['interior/metropolitana']) == count_metropolis['interior/metropolitana'].sum()

True

**Aqui será criado um novo atributo com base na variável 'municipio', removendo a ascentuação, removendo espaços e transoformando os caracteres para minúsculos, pois iremos fazer algumas comparações com a lista de metrópoles acima para imputação dos valores faltantes na variável 'interior/metropolitana'**

In [6]:
cvd_2020_1['temp_municipio'] = cvd_2020_1['municipio'].str.lower()
cvd_2020_1['temp_municipio'] = [re.sub('\W+','',unidecode.unidecode(value)) for value in cvd_2020_1['temp_municipio']] 

**Aqui de fato fazemos a imputação nos valores faltantes levando em consideração o tipo de município e a lista que foi criada com as nomenclaturas das metrópoles**

In [7]:
cvd_2020_1['interior/metropolitana'] = np.where((np.isnan(cvd_2020_1['interior/metropolitana'])) & ~(cvd_2020_1['temp_municipio'].isin(metropolitanas)),0,cvd_2020_1['interior/metropolitana'])
cvd_2020_1['interior/metropolitana'] = np.where((np.isnan(cvd_2020_1['interior/metropolitana'])) & (cvd_2020_1['temp_municipio'].isin(metropolitanas)),1,cvd_2020_1['interior/metropolitana'])

In [15]:
# Remoção do atributo 'temp_municipio'
cvd_2020_1.drop('temp_municipio',axis = 1, inplace=True)
cvd_2020_1.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,0,0,76,0,0,0,2020-02-25,9,210147125,0,0,0,0,0,0,0.0
1,Brasil,0,0,76,0,0,0,2020-02-26,9,210147125,1,1,0,0,1,0,0.0
2,Brasil,0,0,76,0,0,0,2020-02-27,9,210147125,1,0,0,0,1,0,0.0
3,Brasil,0,0,76,0,0,0,2020-02-28,9,210147125,1,0,0,0,0,1,0.0
4,Brasil,0,0,76,0,0,0,2020-02-29,9,210147125,2,1,0,0,1,1,0.0


In [22]:
cvd_2020_1['populacaoTCU2019'].value_counts()

0        2667
5237      635
4573      508
3203      508
11019     508
         ... 
5714      127
13865     127
31603     127
20598     127
5821      127
Name: populacaoTCU2019, Length: 5105, dtype: int64

**Apesar de realizarmos o tratamento dos valores NaN, foi identificado que alguns atributos possuem valores faltantes preenchidos com o valor 0, pensando nisso, foi mapeado e analisado em percentual a quantidade de valores faltantes do nosso dataset**

In [40]:
populacaoTCU2019 = (len(cvd_2020_1.query("populacaoTCU2019 == 0")) / len(cvd_2020_1)) * 100
municipio = (len(cvd_2020_1.query("municipio == '0'")) / len(cvd_2020_1)) * 100
estado = (len(cvd_2020_1.query("estado == '0'")) / len(cvd_2020_1)) * 100
codRegiaoSaude = (len(cvd_2020_1.query("codRegiaoSaude == 0")) / len(cvd_2020_1)) * 100
nomeRegiaoSaude = (len(cvd_2020_1.query("nomeRegiaoSaude == '0'")) / len(cvd_2020_1)) * 100

print(populacaoTCU2019, municipio, estado, codRegiaoSaude, nomeRegiaoSaude)

0.37323249851658624 1.0045230125054578 0.03428644999496199 1.0045230125054578 1.0045230125054578


**Com base nas porcentagens das variáveis analisadas acima, iremos substituir o valor 0 por 'Não informado' nos atributos categóricos e nos valores contínuos manteremos no mesmo formato, pois a remoção dos registros acabam impactando na remoção de valores de outras variáveis com maior relevância na análise**

In [45]:
cvd_2020_1['municipio'].replace('0','Não Informado',inplace=True)
cvd_2020_1['estado'].replace('0','Não Informado',inplace=True)
cvd_2020_1['nomeRegiaoSaude'].replace('0','Não Informado',inplace=True)
cvd_2020_1.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,Não Informado,Não Informado,76,0,0,Não Informado,2020-02-25,9,210147125,0,0,0,0,0,0,0.0
1,Brasil,Não Informado,Não Informado,76,0,0,Não Informado,2020-02-26,9,210147125,1,1,0,0,1,0,0.0
2,Brasil,Não Informado,Não Informado,76,0,0,Não Informado,2020-02-27,9,210147125,1,0,0,0,1,0,0.0
3,Brasil,Não Informado,Não Informado,76,0,0,Não Informado,2020-02-28,9,210147125,1,0,0,0,0,1,0.0
4,Brasil,Não Informado,Não Informado,76,0,0,Não Informado,2020-02-29,9,210147125,2,1,0,0,1,1,0.0


#### Explorando e Tratando os Dados da Covid do período 2020.2

In [47]:
cvd_2020_2.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-08-01,31,210147125.0,2707877,45392,93563,1088,1865729.0,748585.0,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-08-02,32,210147125.0,2733677,25800,94104,541,1883677.0,755896.0,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-08-03,32,210147125.0,2750318,16641,94665,561,1912319.0,743334.0,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-08-04,32,210147125.0,2801921,51603,95819,1154,1970767.0,735335.0,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-08-05,32,210147125.0,2857597,55676,97240,1421,2020637.0,741180.0,NaN
